In [1]:
#两件套
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("../data/breast-cancer-wisconsin-data.csv")  #读取本地数据
#中文列名
ch_name=["样本编码","团块厚度","细胞大小的均匀性","细胞形状的均匀性","边缘附着力",
         "单层上皮细胞大小","裸核","乏味染色质","正常和仁","线粒体","类别"]
data.columns = [x for x in ch_name]  #列名中文化

In [3]:
(data[ch_name]=="?").sum()   #裸核中有异常数据

样本编码         0
团块厚度         0
细胞大小的均匀性     0
细胞形状的均匀性     0
边缘附着力        0
单层上皮细胞大小     0
裸核          16
乏味染色质        0
正常和仁         0
线粒体          0
类别           0
dtype: int64

In [4]:
#数据清洗（删除异常数据）
delindex = data[data["裸核"]=="?"].index  #得到异常值所在行索引
data.drop(delindex,inplace=True)   #删除，覆盖原数据
(data[ch_name]=="?").sum()   #异常数据被删除

样本编码        0
团块厚度        0
细胞大小的均匀性    0
细胞形状的均匀性    0
边缘附着力       0
单层上皮细胞大小    0
裸核          0
乏味染色质       0
正常和仁        0
线粒体         0
类别          0
dtype: int64

In [5]:
X = np.array(data[ch_name[1:10]])   #提取特征
Y = np.array(data[ch_name[10]])    #提取标签

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)  #2：8原则

In [13]:
from sklearn.ensemble import AdaBoostClassifier  #导入AdaBoost分类器
from sklearn.tree import DecisionTreeClassifier #导入决策树分类器
clf = AdaBoostClassifier(
    DecisionTreeClassifier(random_state=66),    #使用决策树分类器作为基础预测器
    n_estimators=200,   #最大迭代次数
    algorithm="SAMME.R",#SAMME即用对样本集分类效果作为弱学习器权重，
                        #而SAMME.R使用了对样本集分类的预测概率大小来作为弱学习器权重。
                        #由于SAMME.R使用了概率度量的连续值，迭代一般比SAMME快。
    learning_rate=0.5,   #每个弱学习器的权重缩减系数
    #loss="linear"       #有线性‘linear’(默认), 平方‘square’和指数 ‘exponential’三种选择
    random_state=66)    #随机种子
clf.fit(X_train,Y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=66),
                   learning_rate=0.5, n_estimators=200, random_state=66)

![jupyte](../机器学习实战(书本笔记)/图片/AdaBoost原理.jpg)

In [16]:
score = clf.score(X_test,Y_test)
score

0.9562043795620438

In [19]:
'''测试集分类结果的整理与打印'''
predict,test=[],[]
class_names={2:"良性",4:"恶性"}

for i in range(len(clf.predict(X_test))):  #预测结果转文字
    predict.append(class_names[clf.predict(X_test)[i]])
    
for i in range(len(Y_test)):     #实际结果转文字
    test.append(class_names[Y_test[i]])
    
pd.concat([pd.DataFrame(X_test,columns=ch_name[1:10]),
           pd.DataFrame(predict,columns=["预测分类"]),
           pd.DataFrame(test,columns=["实际分类"])],
           axis=1)

,团块厚度,细胞大小的均匀性,细胞形状的均匀性,边缘附着力,单层上皮细胞大小,裸核,乏味染色质,正常和仁,线粒体,预测分类,实际分类
0,8,6,4,3,5,9,3,1,1,恶性,恶性
1,2,1,1,1,2,1,2,2,1,良性,良性
2,2,1,1,2,2,1,3,1,1,良性,良性
3,1,1,1,1,2,1,3,1,1,良性,良性
4,1,1,1,1,2,1,2,1,1,良性,良性
...,...,...,...,...,...,...,...,...,...,...,...
132,2,3,1,1,5,1,1,1,1,恶性,良性
133,1,6,8,10,8,10,5,7,1,恶性,恶性
134,1,1,1,1,2,1,1,1,1,良性,良性
135,10,10,10,8,6,1,8,9,1,恶性,恶性
